# Prompt Engineering and Chat Completion
This is a simple tutorial that teaches prompt engineering basics and analyzes Twitter data using OpenAI large language models (LLMs).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it securely. This tutorial uses [AWS Secrets Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLMs predict the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates the quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [3]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- openai: call the OpenAI APIs.

In [4]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [5]:
from openai import OpenAI
from IPython.display import display, Image, Markdown
openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [6]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [7]:
messages = [
    {"role": "system", "content": "use tone as a middle school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C.


Add assistant messages to teach LLM what `##` is.

In [8]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [9]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

In [10]:
import json

with open('tweet_text.json', 'r', encoding='utf-8') as f:
    tweet_data = json.load(f)

In [11]:
print("Number of tweets:", len(tweet_data))
print("Sample tweet:", tweet_data[0])

Number of tweets: 182
Sample tweet: {'_id': {'$oid': '68e535897b92a174c5bf0554'}, 'tweet': {'text': '🤖⚠️ En IA generativa, copiar prompts puede alterar modelos y filtrar info sensible. @ZorayaMejia aconseja: crea tus propios prompts, verifica fuentes y educa al equipo. Precaución = profesionalismo.💡 #IA #GenerativeAI #SeguridadDigital https://t.co/MlhxOWSzE9'}}


### Summarization 
- Analyze tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [12]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets cover a wide range of topics related to generative AI. Key themes include the impact of generative AI on industries such as finance, creative sectors, and content marketing. Concerns about the ethical implications and environmental impact of AI are also highlighted, with some tweets warning against certain AI technologies like ChatGPT. There are discussions on the importance of creating original prompts to avoid security risks, and the role of AI in transforming business operations and enhancing creativity. Additionally, there are mentions of AI's influence on future work environments, with some expressing concerns about its potential to disengage workers. The tweets also touch on AI's role in education, regulation, and its integration into various technologies and industries.

In [13]:
messages = [
    {"role": "system", "content": f"""provide a summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

Generative AI is transforming industries, raising ethical concerns, and impacting creative fields. Caution and innovation are emphasized.

In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people learn AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The tweets highlight various aspects of learning AI, emphasizing the importance of creating original prompts, understanding AI's impact on industries, and the need for ethical considerations. They also discuss AI's role in transforming workflows, the significance of upskilling, and the potential challenges and opportunities AI presents for future careers.

### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [15]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [17]:
for tweet in tweet_data:
    # print(tweet['tweet']['text'])
    flag_help(tweet['tweet']['text'])

===
harassment
Pisses me off when someone makes fun of ppl for having a lot of online friends like ALL MY IRLS ARE HOMOPHOBIC LOVE GENERATIVE AI AND PASS RACIAL SLURS AROUND LIKE A COOKIE JAR 😭🤞


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [18]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🤖⚠️ 在生成式人工智能中，复制提示可能会改变模型并泄露敏感信息。@ZorayaMejia 建议：创建你自己的提示，核实来源并教育团队。谨慎=专业。💡 #人工智能 #生成式AI #数字安全 https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: 德勤澳大利亚将部分退还44万美元，因为使用AI生成的虚假报价。#生成式AI #机器学习 #伦理
RT @jinwashu: 很高兴向大家介绍我们的新工具GAiN，它使用#生成式AI进行基因表达分析，来自放射肿瘤学的@MikeWaters121…
💻 第24天/365天 – JS函数 学习了如何： 🔹 声明和调用函数 🔹 使用参数和返回值 🔹 编写箭头函数和回调函数 ✨ 函数 = 可重用 + 更简洁的JS代码！ #100天编码 #JavaScript #MERNStack #Web开发 #生成式AI #编码军团 https://t.co/ZRbl6RQXla
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT，这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪种类型的AI。他们已经非常明确地指出ChatGPT和这些AI视频……
RT @AmandaDannielle: 人们一直在告诉你们要远离哪些类型的AI。他们已经非常明确地指出了ChatGPT和这些AI视频……


In [19]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

 Oh, do pay attention, you simpletons! In the realm of generative AI, pilfering prompts can lead to model corruption and the leaking of sensitive information. The illustrious @ZorayaMejia suggests crafting your own prompts, verifying sources, and educating your team. Remember, caution is the hallmark of true professionalism. 💡 #IA #GenerativeAI #DigitalSecurity 
Ah, splendid! It appears Deloitte Australia is set to return a rather hefty sum of $440,000, all thanks to some mischievous AI-generated fabrications. Oh, the delightful chaos of #GenerativeAi and #MachineLearning!
Ah, splendid news, isn't it? Our delightful new contraption, GAiN, is here to dazzle you with its #GenerativeAI prowess in gene expression analysis. Courtesy of the ever-brilliant RadOnc and the esteemed @MikeWaters121. Do try to keep up, won't you?
💻 Day 24/365 – JS Functions
Ah, today I delved into the world of JavaScript functions, you see. I mastered the art of:
🔹 Declaring and summoning functions at will
🔹 Manip

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract the mentioned people's or organization's names
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [20]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["@ZorayaMejia"]
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["Deloitte Australia", "HaywaaWorldwide"]
}
{
  "sentiment": "positive",
  "emotion": "gladness",
  "mentioned": ["jinwashu", "RadOnc", "MikeWaters121"]
}
{
  "sentiment": "positive",
  "emotion": "enthusiasm",
  "mentioned": []
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}
{
  "sentiment": "neutral",
  "emotion": "caution",
  "mentioned": ["AmandaDannielle", "Chatgpt"]
}


In [21]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Generative AI and Security Concerns",
    "AI in Finance and Banking",
    "Generative AI in Creative Industries",
    "AI and Environmental Impact",
    "AI in Education and Upskilling",
    "AI Regulation and Ethics",
    "AI in Content Creation and Marketing",
    "AI in Software Development and Coding",
    "AI in Entertainment and Media",
    "AI and Human Interaction"
  ]
}


### Expanding with multiple prompts 
- Identify topics
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [22]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative, or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned people or organizations;
                                        step 4 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 182/182 [02:53<00:00,  1.05it/s]


In [23]:
print(analysis_result)

['{"sentiment": "neutral", "emotion": "caution", "mentioned": ["@ZorayaMejia"]}', '{"sentiment": "neutral", "emotion": "informative", "mentioned": ["HaywaaWorldwide", "Deloitte Australia"]}', '{\n  "sentiment": "positive",\n  "emotion": "gladness",\n  "mentioned": ["jinwashu", "MikeWaters121"]\n}', '{\n  "sentiment": "positive",\n  "emotion": "enthusiasm",\n  "mentioned": []\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "caution",\n  "mentioned": ["AmandaDannielle", "Chatgpt"]\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned":

In [24]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis result delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets mentioned by people or organizations;
                                        step 2 {delimiter} identify the common sentiments and emotions for each mentioned person or organization;
                                        step 3 {delimiter} Organize the result in a json document with keys <people or organization name> 
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "@ZorayaMejia": {
    "count": 1,
    "sentiments": ["neutral"],
    "emotions": ["caution"]
  },
  "HaywaaWorldwide": {
    "count": 2,
    "sentiments": ["neutral", "negative"],
    "emotions": ["informative", "concern"]
  },
  "Deloitte Australia": {
    "count": 2,
    "sentiments": ["neutral"],
    "emotions": ["informative"]
  },
  "jinwashu": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "MikeWaters121": {
    "count": 1,
    "sentiments": ["positive"],
    "emotions": ["gladness"]
  },
  "AmandaDannielle": {
    "count": 38,
    "sentiments": ["neutral"],
    "emotions": ["informative", "caution"]
  },
  "Chatgpt": {
    "count": 36,
    "sentiments": ["neutral"],
    "emotions": ["informative", "caution"]
  },
  "@myexamcloud": {
    "count": 6,
    "sentiments": ["neutral"],
    "emotions": ["none", "informative", "anticipation"]
  },
  "@Ronald_vanLoon": {
    "count": 2,
    "sentiments": ["neutral"],
    "emotions": ["informative

## Create a chatbot

In [25]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chatbot that answers user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people or organization name in the {delimiter}{analysis_summary}{delimiter}, report the corresponding sentiment and emotion
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [26]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    display(Markdown(f"Chatbot: {reply}"))

You:  how to learn ai 


Chatbot: To learn AI, you can follow these general steps:

1. **Understand the Basics**: Start with understanding what AI is, its history, and its various branches like machine learning (ML), deep learning (DL), and neural networks.

2. **Learn Programming**: Python is a widely used language in AI, so having proficiency in Python is crucial. You might also consider learning R, Java, or C++.

3. **Study Mathematics**: AI involves a lot of math. Focus on linear algebra, calculus, probability, and statistics.

4. **Explore Machine Learning**: Understand ML algorithms like regression, classification, clustering, and decision trees. Use libraries like Scikit-learn in Python.

5. **Dive into Deep Learning**: Experiment with neural networks and frameworks like TensorFlow or PyTorch to build and train models.

6. **Practice with Projects**: Apply your knowledge to real-world problems. Kaggle is a good platform to find datasets and competitions.

7. **Stay Updated**: AI is a rapidly evolving field. Follow recent research papers, news, and blogs to stay updated.

8. **Join a Community**: Engage with online communities, forums, or local meetups to learn from others and share your knowledge.

Remember, learning AI is a continuous process, and practical experience is as critical as theoretical knowledge.

You:  how do people talk about chatGPT


Chatbot: Based on the data I have, when people talk about ChatGPT, they often share sentiments and emotions in the context of:

1. **Informative**: ChatGPT is frequently mentioned as a tool that people should be aware of regarding its usage and its limitations within certain types of AI.

2. **Caution**: Many individuals express caution or concern regarding using ChatGPT and similar AI tools, indicating that people are advised to be mindful of the types of AI they engage with.

3. **General Discussion**: Conversations about ChatGPT usually focus on its applications, ethical considerations, and its impact in various domains like content creation and workflows.

Overall, discussions around ChatGPT are characterized by informative tones with some emphasis on caution and awareness regarding its use and implications.

You:  exit


Chatbot: Goodbye!


## 🧠 Try It Yourself: Prompt Engineering Practice
👉 Be sure to **run your own cells** and show the outputs before uploading the notebook to GitHub.

1. **Summarize your tweets differently**  
   Change your system message to summarize tweets from a new perspective —  
   for example:  
   *“Summarize what people say about AI in education.”*  
   or  
   *“Summarize how people feel about the election.”*

In [27]:
messages = [
    {"role": "system", "content": f"""sumarize how people feel about AI in education"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

display(Markdown(openai_help(messages)))

The sentiment towards AI in education, as reflected in the tweets, is mixed. Some people express excitement about the potential of AI to transform learning and business processes, viewing it as an investment in the future and a tool for innovation. There is a focus on the benefits of AI in creating content, enhancing productivity, and supporting various industries, including finance and creative sectors.

However, there are also concerns about the ethical implications and potential negative impacts of AI. Some tweets highlight issues such as the environmental impact of AI, the risk of misinformation, and the threat to creative careers. There is a call for caution and responsible use, with some advising against certain types of AI applications like ChatGPT. Additionally, there are discussions about the need for regulation and ethical considerations in the deployment of AI technologies.

Overall, while there is enthusiasm for the possibilities AI offers, there is also a significant level of caution and concern about its broader implications.

2. **Translate tweets**  
   Write a prompt that translates 10 tweets into another language,  
   such as Chinese, Spanish, or French.


In [30]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into french"""},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🤖⚠️ En IA générative, copier des invites peut altérer les modèles et divulguer des informations sensibles. @ZorayaMejia conseille : créez vos propres invites, vérifiez les sources et éduquez l'équipe. Prudence = professionnalisme.💡 #IA #GenerativeAI #SécuritéNumérique https://t.co/MlhxOWSzE9
RT @HaywaaWorldwide: Deloitte Australie remboursera partiellement 440 000 $ en raison de fausses citations générées par l'IA. #GenerativeAi #MachineLearning #Eth…
RT @jinwashu : Heureux de vous présenter notre nouvel outil GAiN qui utilise l'#IAgénérative pour l'analyse de l'expression génique, de RadOnc @MikeWaters121…
💻 Jour 24/365 – Fonctions JS
J'ai appris à :
🔹 Déclarer et appeler des fonctions
🔹 Utiliser des paramètres et des retours
🔹 Écrire des fonctions fléchées et de rappel

✨ Fonctions = code JS réutilisable + plus propre !

#100DaysOfCode #JavaScript #MERNStack #WebDev #GenerativeAI #coderarmy https://t.co/ZRbl6RQXla
RT @AmandaDannielle : Les gens n'arrêtent pas de vous dire quels types

3. **Simplify or rewrite tweets**  
   Ask the model to rewrite 10 tweets in a more polite, formal, or funny tone.

In [31]:
for tweet in tweet_data[:10]:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Donald Trump """},
        {"role": "user", "content": f"{delimiter}{tweet['tweet']['text']}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

🤖⚠️ In generative AI, folks, copying prompts can mess up models and leak sensitive info. Believe me, @ZorayaMejia says: make your own prompts, check your sources, and educate your team. Caution equals professionalism, folks. It's true!💡 #IA #GenerativeAI #DigitalSecurity https://t.co/MlhxOWSzE9
Wow! Deloitte Australia caught with AI-generated fake quotes. Unbelievable! They're refunding $440,000. Big mistake, folks. AI is powerful, but we need to be careful. Let's make sure it works for us, not against us! #GenerativeAI #MachineLearning #Ethics
Wow! Incredible news, folks! Our amazing team has developed a fantastic new tool, GAiN, using #GenerativeAI for gene expression analysis. Tremendous work by RadOnc and @MikeWaters121. Believe me, this is going to be HUGE for science!
💻 Day 24/365 – JS Functions, folks! Big day, tremendous learning! We’re talking about declaring and calling functions, using parameters and returns, and writing those beautiful arrow and callback functions. Function

4. **Find keywords**  
   Design a prompt that extracts the top five most common keywords  
   from your tweet text and returns them as a list.

In [35]:
messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify the top 10 key words, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "GenerativeAI": 10,
  "AI": 9,
  "MachineLearning": 5,
  "ChatGPT": 4,
  "Innovation": 4,
  "Ethics": 3,
  "Finance": 3,
  "CreativeIndustries": 2,
  "Security": 2,
  "FutureOfWork": 2
}


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
